In [1]:
from scipy import sparse
import numpy as np
from timeit import default_timer as timer
import copy

In [2]:
def makeM (l, g, b, a):
    #lil is faster for writing
    m = sparse.lil_matrix(l)
    for i in range(a[0].shape[0]):
        m[a[0][i], a[1][i]] = b/g[a[1][i]]
    for i in range(l.shape[1]):
        if g[i] == 0:
            m[:, i] = b/l.shape[1]
    return m

In [3]:
def makeG(L):
    g = np.zeros(L.shape[1])
    a = sparse.csr_matrix.nonzero(L)
    unique, counts = np.unique(a[1], return_counts=True)
    for i in range(unique.shape[0]):
        g[unique[i]] = counts[i]
    return g

In [4]:
def vbuild(w, b, n):
    #for whatever reason it changed the original vector, so I used deepcopy
    k = copy.deepcopy(w)
    k[:] = np.sum(w) * (1 - b) / n
    return k

In [5]:
def dataset2csr(filename, nodes, edges):
    #Changed TRUE to 1., it was easier with float
    rows = []; cols = []    
    with open(filename, 'r') as f:
        for line in f.readlines()[4:]:
            o, d = (int(x)-1 for x in line.split())
            rows.append(d)
            cols.append(o)
    return(sparse.csr_matrix(([1.]*edges, (rows, cols)), shape=(nodes, nodes)))

In [6]:
#consts for the first file
n = 5
ed = 8
b = 0.8

In [7]:
l = dataset2csr(filename='/Users/markblumenau/Downloads/miniGOOGLE.txt', nodes = n, edges = ed)
#check whether I went crazy or everything is fine, not doing that for bigboi
print(l) 
a = sparse.csr_matrix.nonzero(l)
L = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        L[i, j] = l[i, j]
print(L)

  (1, 0)	1.0
  (1, 2)	1.0
  (2, 0)	1.0
  (2, 1)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (4, 1)	1.0
  (4, 3)	1.0
[[0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 1. 0. 1. 0.]]


In [8]:
gg = makeG(l)
#check again, not doing that for bigboi
print(gg)
G = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        if gg[i] != 0:
            if L[j, i]:
                G[j, i] = 1/gg[i]
        else:
            G[j, i] = 1/n
print(G)

[4. 2. 1. 1. 0.]
[[0.   0.   0.   0.   0.2 ]
 [0.25 0.   1.   0.   0.2 ]
 [0.25 0.5  0.   0.   0.2 ]
 [0.25 0.   0.   0.   0.2 ]
 [0.25 0.5  0.   1.   0.2 ]]


In [9]:
v = np.random.random(n)
v /= np.sqrt(v@v)
vp = np.zeros(n)
m = makeM(l, gg, b, a)
#Another check
M = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        M [i, j] = m[i, j]
print(M)
i = 0
while np.linalg.norm(v-vp, 1) > 1.0E-4:
    i += 1
    vp = v
    v = m @ vp  + vbuild(vp, b, l.shape[1])
    v /= np.sqrt(v@v)
    
print(i, "ITER NUM")
print ("1 is the eigval for that vector")
print(v)
#check again, must be approx. 1
print((m @ v + vbuild(v, b, l.shape[1])) / v) 

[[0.   0.   0.   0.   0.16]
 [0.2  0.   0.8  0.   0.16]
 [0.2  0.4  0.   0.   0.16]
 [0.2  0.   0.   0.   0.16]
 [0.2  0.4  0.   0.8  0.16]]
18 ITER NUM
1 is the eigval for that vector
[0.18136346 0.57609822 0.4480631  0.21763486 0.62216771]
[0.99998051 0.99997375 1.00001966 0.9999897  1.00001943]


In [10]:
print(v.argmax()+1, "is VIP, Very Important Page")

5 is VIP, Very Important Page


Ready for the bigboi, hopefully it will take less than 5 hours

In [11]:
#consts for the bigboi
n = 281903
ed = 2312497
b = 0.8

In [12]:
e = timer()
l = dataset2csr(filename='/Users/markblumenau/Downloads/web-Stanford.txt', nodes = n, edges = ed)
gg = makeG(l)
v = np.random.random(n)
v /= np.sqrt(v@v)
i = 0
vp = np.zeros(n)
a = sparse.csr_matrix.nonzero(l)
m = makeM(l, gg, b, a)
print("Part 1 time:", timer() - e)
t = timer() - e

Part 1 time: 28.675953679


In [13]:
q = timer()
while np.linalg.norm(v-vp, 1) > 1.0E-4:
    i += 1
    vp = v
    v = m @ vp  
    v += vbuild(vp, b, l.shape[1])
    v /= np.sqrt(v@v)
print(i, "ITER NUM")
print ("1 is the eigval for that vector")
print(v.argmax()+1, "is VIP, Very Important Page")
#check
print((m @ v + vbuild(v, b, l.shape[1])) / v) 
#time
print ("Part 2 time:", timer() - q)
t += timer() - q
print("Total time:", t)

45 ITER NUM
1 is the eigval for that vector
89073 is VIP, Very Important Page
[1.         0.9999826  0.99999997 ... 1.         0.99999982 0.99999997]
Part 2 time: 61.39274669700001
Total time: 90.068959817


Вероятно, можно добиться лучшего результата, но я уже не знаю как. Каждое домножение матрицы на вектор само по себе занимает 1.3 секунды, а как это делать быстрее numpy я не придумал (оно, судя по разным темам на форумах, на C++/Fortran).